[View in Colaboratory](https://colab.research.google.com/github/aaronpp65/Handrwitten_digit_classification/blob/master/handwritten_digits_NN.ipynb)

In [0]:
from __future__ import print_function

import glob
import math
import os

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows=10
pd.options.display.float_format='{:.1f}'.format


In [0]:
mnist_dataframe=pd.read_csv("https://storage.googleapis.com/mledu-datasets/mnist_train_small.csv",sep=",",header=None)

mnist_dataframe = mnist_dataframe.head(10000)

mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))


In [0]:
#Create labels and features
def labels_and_features(dataset):
	
	labels = dataset[0]
	
	features = dataset.loc[:,1:784]
	features = features/255
	
	return labels, features
	
training_targets,training_examples = labels_and_features(mnist_dataframe[:7500])

validation_targets,validation_examples = labels_and_features(mnist_dataframe[7500:10000])

In [0]:
#Create input function for training
def train_input_fn(features,labels,batch_size,num_epochs = None,shuffle = True):
  
  def _input_fn(num_epochs=None, shuffle=True):
    
    idx = np.random.permutation(features.index)
    raw_features = {"pixels":features.reindex(idx)}
    raw_targets = np.array(labels[idx])
    
    ds = Dataset.from_tensor_slices((raw_features,raw_targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
      ds = ds.shuffle(10000)
      
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch
  return _input_fn

In [0]:
#Create input function for Prediction

def predict_input_fn(features, labels, batch_size):
   
  def _input_fn():
    
    raw_features = {"pixels": features.values}
    raw_targets = np.array(labels)
    
    ds = Dataset.from_tensor_slices((raw_features, raw_targets))
    ds = ds.batch(batch_size)
    
        
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
def train_NN(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  
 

  periods = 10
 
  steps_per_period = steps / periods  
  
  predict_training_input_fn = predict_input_fn(training_examples, training_targets, batch_size)
  
  predict_validation_input_fn = predict_input_fn(validation_examples, validation_targets, batch_size)
  
  training_input_fn = train_input_fn(training_examples, training_targets, batch_size)
  
#creating feature columns
  
  feature_columns = [tf.feature_column.numeric_column('pixels', shape=784)]
  
#creating NN
  my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  classifier = tf.estimator.DNNClassifier(
      feature_columns=feature_columns,
      n_classes=10,
      hidden_units=hidden_units,
      optimizer=my_optimizer,
      config=tf.contrib.learn.RunConfig(keep_checkpoint_max=1)
  )

#Training 
  print("Training model...")
  print("LogLoss error (on validation data):")
  training_errors = []
  validation_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
  
    # Take a break and compute probabilities.
    training_predictions = list(classifier.predict(input_fn=predict_training_input_fn))
    training_probabilities = np.array([item['probabilities'] for item in training_predictions])
    training_pred_class_id = np.array([item['class_ids'][0] for item in training_predictions])
    training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,10)
        
    validation_predictions = list(classifier.predict(input_fn=predict_validation_input_fn))
    validation_probabilities = np.array([item['probabilities'] for item in validation_predictions])    
    validation_pred_class_id = np.array([item['class_ids'][0] for item in validation_predictions])
    validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,10)    
    
    # Compute training and validation errors.
    training_log_loss = metrics.log_loss(training_targets, training_pred_one_hot)
    validation_log_loss = metrics.log_loss(validation_targets, validation_pred_one_hot)
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, validation_log_loss))
    # Add the loss metrics from this period to our list.
    training_errors.append(training_log_loss)
    validation_errors.append(validation_log_loss)
  print("Model training finished.")
  # Remove event files to save disk space.
  _ = map(os.remove, glob.glob(os.path.join(classifier.model_dir, 'events.out.tfevents*')))
  
  # Calculate final predictions (not probabilities, as above).
  final_predictions = classifier.predict(input_fn=predict_validation_input_fn)
  final_predictions = np.array([item['class_ids'][0] for item in final_predictions])
  
  
  accuracy = metrics.accuracy_score(validation_targets, final_predictions)
  print("Final accuracy (on validation data): %0.2f" % accuracy)

  

  return classifier

In [39]:
classifier = train_NN(
    learning_rate=0.05,
    steps=1000,
    batch_size=30,
    hidden_units=[100, 100],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets) 

Training model...
LogLoss error (on validation data):
  period 00 : 4.59
  period 01 : 3.34
  period 02 : 3.85
  period 03 : 2.65
  period 04 : 2.62
  period 05 : 2.79
  period 06 : 2.04
  period 07 : 1.96
  period 08 : 1.98
  period 09 : 1.96
Model training finished.
Final accuracy (on validation data): 0.94


In [65]:
#Testing

mnist_test_dataframe = pd.read_csv(
  "https://storage.googleapis.com/mledu-datasets/mnist_test.csv",
  sep=",",
  header=None)

test_targets, test_examples = labels_and_features(mnist_test_dataframe)
predict_test_input_fn = predict_input_fn(test_examples, test_targets, batch_size=100)

test_predictions = classifier.predict(input_fn=predict_test_input_fn)
test_predictions = np.array([item['class_ids'][0] for item in test_predictions])
  
accuracy = metrics.accuracy_score(test_targets, test_predictions)
print("Accuracy on test data: %0.2f \n" % accuracy)

rand_example = np.random.choice(test_examples.index)

print("Random handwritten digit : %1d \n" % test_targets.values[rand_example])
print("Prediction : %1d" % test_predictions[rand_example])


Accuracy on test data: 0.95 

Random handwritten digit 9 

Prediction 9
